This notebook checks the behaviour of the embedding space

In [1]:
import re
import os
import sys
import pickle
import random
import glob
import scipy.sparse

import numpy as np
import pandas as pd

from annoy import AnnoyIndex
from scipy.spatial import distance

scriptpath = "../../"
sys.path.append(os.path.abspath(scriptpath))
from helpers.helpers import *

In [2]:
COMMON_PATH = '/dlabdata1/youtube_large/jouven/word2vecf_preprocessing'

In [6]:
'''
Retrieve the array obtained by apllying the dimentionality reduction algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETER:
    - file_path: the path where the embedding graph is stored

RETURN: 
    - df: DataFrame representing the graph in the embedding space
'''
def get_dataframe_in_embedding_space(filename):
    graph_matrix = np.load(os.path.join(COMMON_PATH, filename + '.npy'))
    df = pd.DataFrame(graph_matrix)
    df = df.rename(lambda x: 'dr'+str(x), axis='columns')
    df['index'] = np.load(os.path.join(COMMON_PATH, filename + 'index_ordering.npy')).astype(int)
    df = df.sort_values(by=['index'])
    return df.set_index('index')


In [7]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

### Find k closest channels using annoy library

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we should get similar channels if the embedding space is good.

In [7]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/channelcrawler.csv")
channelcrawler['channel_id'] = channelcrawler['link'].str.split('/').str[-1]
channelcrawler.head()

,category,join_date,link,name,subscribers,videos,channel_id
0,Film and Animation,2017-05-21,http://www.youtube.com/channel/UCBJuEqXfXTdcPS...,MagnusNation,65100,28,UCBJuEqXfXTdcPSbGO9qqn1g
1,Entertainment,2011-12-13,http://www.youtube.com/channel/UCkNW9Q1VR_aeZ6...,Mago Dario Animazion...,60200,48,UCkNW9Q1VR_aeZ6uht83jJVQ
2,Music,2013-09-13,http://www.youtube.com/channel/UC1xcnrpcF59FWW...,Mägo de Oz - Topic,40200,395,UC1xcnrpcF59FWWELtZvJTdg
3,Music,2008-03-17,http://www.youtube.com/channel/UCXhkGgooXHDNwg...,Mago Merlino,14800,838,UCXhkGgooXHDNwgJXmoTSN7g
4,Entertainment,2014-10-19,http://www.youtube.com/channel/UCvZGsuvKlYOGiZ...,MAGO TOMÁS,26200,31,UCvZGsuvKlYOGiZTsxwJNS5Q


In [8]:
channelcrawler = channelcrawler[channelcrawler['channel_id'].apply(lambda row: row in channels_id)]

In [9]:
channelcrawler['category'].unique()

array(['Comedy', 'Gaming', 'Film and Animation', 'Science & Technology',
       'Music', 'Education', 'Sports', 'People & Blogs', 'Entertainment',
       'Nonprofits & Activism', 'News & Politics', 'Travel & Events',
       'Howto & Style', 'Autos & Vehicles', 'Pets & Animals', nan],
      dtype=object)

In [10]:
channelcrawler[channelcrawler['category'] == 'Film and Animation'][50:100]

,category,join_date,link,name,subscribers,videos,channel_id
3767,Film and Animation,2015-11-13,http://www.youtube.com/channel/UCWkfvm1YaPY8H1...,IAmZamber Reacts,44900,510,UCWkfvm1YaPY8H1rQY6fB60Q
3787,Film and Animation,2015-03-08,http://www.youtube.com/channel/UCBQhLER2rVxN2T...,MSK Movies,1080000,4166,UCBQhLER2rVxN2TkSuPfJ6rA
3791,Film and Animation,2016-07-08,http://www.youtube.com/channel/UC_CstbFmNkvWAH...,Aluminium Plays,45900,50,UC_CstbFmNkvWAH3RLdxAusw
3799,Film and Animation,2012-02-12,http://www.youtube.com/channel/UCpW-R7T0kA4jFa...,Aluriya,39400,46,UCpW-R7T0kA4jFaB-GAQurXQ
3865,Film and Animation,2014-03-07,http://www.youtube.com/channel/UCSZLzCcpZqWJUx...,V KINGS,114000,300,UCSZLzCcpZqWJUxUi8h57sew
3954,Film and Animation,2017-03-18,http://www.youtube.com/channel/UCmlU7184B6TBOy...,Bonda,173000,73,UCmlU7184B6TBOyJ2OhseCAg
3976,Film and Animation,2012-12-30,http://www.youtube.com/channel/UCDzMRfmdypVUxZ...,PushedToInsanity,531000,629,UCDzMRfmdypVUxZ0H8xvGDOA
3996,Film and Animation,2017-07-25,http://www.youtube.com/channel/UCLiVYcPG3bsVAw...,ohitscosmo,81500,49,UCLiVYcPG3bsVAwqhfTsr5kA
4015,Film and Animation,2010-04-13,http://www.youtube.com/channel/UCbsfyGlrjrKQC0...,Jared Owen,339000,47,UCbsfyGlrjrKQC0gbzK0-EiA
4074,Film and Animation,2015-07-14,http://www.youtube.com/channel/UC6DKFSj5aDJeMz...,Nebula Effect,119000,83,UC6DKFSj5aDJeMz0kmbbpu3w


In [8]:
'''
    Retrieve the array obtained by apllying the dimentinality reductin algorithm
    graph_matrix: SHAPE: (channels, n_comp)
    
    PARAMETERS:
        - df_embedding: DataFrame representing the graph in the embedding space
        
    RETURN: The annoy index
    '''
def get_annoy_index(df):
    index = AnnoyIndex(df.shape[1], "euclidean")  # Length of item vector that will be indexed
    df.apply(lambda row: index.add_item(row.name, np.array(row)), axis = 1)
    index.build(100) # 100 trees
    return index

In [9]:
def get_k_nearest_neighbors(path, ref_index_channel, k = 20):
    df = get_dataframe_in_embedding_space(path)
    index = get_annoy_index(df)
    nearest_neighbors_index = index.get_nns_by_item(ref_index_channel, k,)
    nearest_neighbors_id = [dict_ind_channel[val] for val in nearest_neighbors_index]
    return nearest_neighbors_id

In [13]:
dict_channel_ind['UCLFW3EKD2My9swWH4eTLaYw']

19769

In [14]:
filename = 'channel_embedding_30M_dim_400'

In [15]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], k= 40)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCMpOz2KEfkSdd5JeIJh_fxw,Howto & Style,2009-01-18,http://www.youtube.com/channel/UCMpOz2KEfkSdd5...,Shaaanxo,3215340,1422
1,UCXbQzhqSvgVZTUyi1T4AU3w,Howto & Style,2011-08-31,http://www.youtube.com/channel/UCXbQzhqSvgVZTU...,Lauren Curtis,3530000,379
2,UCPRlGA2w7C_DVw-1ynolJYw,People & Blogs,2011-07-19,http://www.youtube.com/channel/UCPRlGA2w7C_DVw...,Shaaanxo Vlogs,555112,742
3,UCLFW3EKD2My9swWH4eTLaYw,Howto & Style,2012-03-06,http://www.youtube.com/channel/UCLFW3EKD2My9sw...,Chloe Morello,2700000,518
4,UC21yq4sq8uxTcfgIxxyE9VQ,Howto & Style,2011-06-27,http://www.youtube.com/channel/UC21yq4sq8uxTcf...,Carli Bybel,6190000,546
5,UCz0Qnv6KczUe3NH1wnpmqhA,Howto & Style,2006-11-27,http://www.youtube.com/channel/UCz0Qnv6KczUe3N...,Nicole Guerriero,2870000,664
6,UCE9_EMdxq6C8giPVpAWkJFQ,Entertainment,2011-08-10,http://www.youtube.com/channel/UCE9_EMdxq6C8gi...,BRITTNEYLEESAUNDERS,1100000,584
7,UC0qI3HpiBua75glb4RV5mWA,Howto & Style,2013-02-21,http://www.youtube.com/channel/UC0qI3HpiBua75g...,Samantha Ravndahl,967843,322
8,UCG9nVSLp4nQlW79sY5ihRrg,Howto & Style,2013-08-17,http://www.youtube.com/channel/UCG9nVSLp4nQlW7...,Desi Perkins,3290000,359
9,UC6jgzx2g3nlbaYkd8EMweKA,Howto & Style,2010-09-07,http://www.youtube.com/channel/UC6jgzx2g3nlbaY...,Jaclyn Hill,5890000,344


In [16]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCAql2DyGU2un1Ei2nMYsqOA'], k= 300)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)

In [17]:
nearest_neighbors[:50]

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAql2DyGU2un1Ei2nMYsqOA,News & Politics,2015-03-17,http://www.youtube.com/channel/UCAql2DyGU2un1E...,Donald J Trump,189000,222
1,UCqC9wT9TI5Va0yWG77-q3IA,News & Politics,2007-06-18,http://www.youtube.com/channel/UCqC9wT9TI5Va0y...,seattletimesdotcom,29100,3588
2,UCjaeHZ3XKFe5U2pr3V7G2xQ,Entertainment,2007-01-20,http://www.youtube.com/channel/UCjaeHZ3XKFe5U2...,MAGAtruthChannel,43300,39
3,UCbBm6SZ235HFxwVKC7Po5IA,News & Politics,2014-02-04,http://www.youtube.com/channel/UCbBm6SZ235HFxw...,News2Share,31100,1117
4,UCgBVz3EzHYKpPS0BITvLRsg,News & Politics,2006-06-27,http://www.youtube.com/channel/UCgBVz3EzHYKpPS...,kxan,65900,62800
5,UC1Z1i5ClfrJhyu9z1hlmDkQ,News & Politics,2015-05-14,http://www.youtube.com/channel/UC1Z1i5ClfrJhyu...,Daily Kos,10800,878
6,UCp7zzDMR-77qQxqG_7_aMfQ,Entertainment,2016-09-22,http://www.youtube.com/channel/UCp7zzDMR-77qQx...,The Redneck Patriot,38900,486
7,UChbYn187BlBxq5LfGqZSv_A,News & Politics,2014-10-13,http://www.youtube.com/channel/UChbYn187BlBxq5...,syracuse.com,27700,6137
8,UCa2gbBXGeGh-Pdtugdg_qcQ,News & Politics,2012-01-26,http://www.youtube.com/channel/UCa2gbBXGeGh-Pd...,The Western Journal,82400,1154
9,UCi4fcBVyo4CAnmdgXeO-NvA,News & Politics,2014-02-27,http://www.youtube.com/channel/UCi4fcBVyo4CAnm...,CBS Boston,63500,31634


In [18]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UC9pXxdNqCc2zjgRXSoowNNg'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UC9pXxdNqCc2zjgRXSoowNNg,News & Politics,2014-07-09,http://www.youtube.com/channel/UC9pXxdNqCc2zjg...,A Blue Dot In Texas,10600,646
1,UCqSR91bTcQyrSZpUdGpJy1w,News & Politics,2008-12-31,http://www.youtube.com/channel/UCqSR91bTcQyrSZ...,Julia Blog,10300,376
2,UCRPdcY4_Al8TrRtgkMzkrQQ,Entertainment,2014-03-21,http://www.youtube.com/channel/UCRPdcY4_Al8TrR...,Tony Dortie - 24/7 E...,98700,333
3,UCXEjC-1iUa1WhwrBp0eBBuw,Entertainment,2016-10-13,http://www.youtube.com/channel/UCXEjC-1iUa1Whw...,Be Less Stupid,52100,309
4,UCWCtoOXSwTcbVjOiK8iFGUg,People & Blogs,2015-06-10,http://www.youtube.com/channel/UCWCtoOXSwTcbVj...,Washington Watch,25000,164
5,UCuDv5p8E-evaRSh542hDV5g,News & Politics,2015-06-16,http://www.youtube.com/channel/UCuDv5p8E-evaRS...,Robert Reich,149000,221
6,UCwlHzGKZA-e2GeLMjIE51uw,News & Politics,2017-08-16,http://www.youtube.com/channel/UCwlHzGKZA-e2Ge...,Parody Project,50400,99
7,UC6ZFN9Tx6xh-skXCuRHCDpQ,News & Politics,2009-11-30,http://www.youtube.com/channel/UC6ZFN9Tx6xh-sk...,PBS NewsHour,1380000,26035
8,UC1Z1i5ClfrJhyu9z1hlmDkQ,News & Politics,2015-05-14,http://www.youtube.com/channel/UC1Z1i5ClfrJhyu...,Daily Kos,10800,878
9,UCriIuQZpMi6gEt_2P7xKCww,News & Politics,2017-12-06,http://www.youtube.com/channel/UCriIuQZpMi6gEt...,Andrew Yang for Pres...,90761,79


In [19]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCAuUUnT6oDeKwE6v1NGQxug'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAuUUnT6oDeKwE6v1NGQxug,People & Blogs,2006-12-06,http://www.youtube.com/channel/UCAuUUnT6oDeKwE...,TED,14800000,3105
1,UCsT0YIqwnpJCM-mx7-gSA4Q,Nonprofits & Activism,2009-06-23,http://www.youtube.com/channel/UCsT0YIqwnpJCM-...,TEDx Talks,20700000,141627
2,UCvQECJukTDE2i6aCoMnS-Vg,Education,2006-10-01,http://www.youtube.com/channel/UCvQECJukTDE2i6...,Big Think,2650000,11135
3,UCHnyfMqiRRG1u-2MsSQLbXA,Education,2010-07-21,http://www.youtube.com/channel/UCHnyfMqiRRG1u-...,Veritasium,6370000,280
4,UC7IcJI8PUf5Z3zKxnZvTBog,Education,2010-05-18,http://www.youtube.com/channel/UC7IcJI8PUf5Z3z...,The School of Life,4870000,724
5,UCqnbDFdCpuN8CMEg0VuEBqA,News & Politics,2006-10-13,http://www.youtube.com/channel/UCqnbDFdCpuN8CM...,The New York Times,2150000,9619
6,UCftwRNsjfRo08xYE31tkiyw,Science & Technology,2005-09-23,http://www.youtube.com/channel/UCftwRNsjfRo08x...,WIRED,5100000,2902
7,UCsXVk37bltHxD1rDPwtNM8Q,Education,2013-07-09,http://www.youtube.com/channel/UCsXVk37bltHxD1...,Kurzgesagt – In a ...,9630035,105
8,UC6107grRI4m0o2-emgoDnAA,Science & Technology,2006-04-04,http://www.youtube.com/channel/UC6107grRI4m0o2...,SmarterEveryDay,7270000,309
9,UCK7tptUDHh-RYDsdxO1-5QQ,News & Politics,2007-06-18,http://www.youtube.com/channel/UCK7tptUDHh-RYD...,Wall Street Journal,1630000,25937


In [20]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCwTkM6CvIsYFaFiMKIKCqHw'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCwTkM6CvIsYFaFiMKIKCqHw,Film and Animation,2005-11-27,http://www.youtube.com/channel/UCwTkM6CvIsYFaF...,James Bond 007,115000,407
1,UCwSIJCMWZC5GDM59wj7pMsg,Entertainment,2006-05-21,http://www.youtube.com/channel/UCwSIJCMWZC5GDM...,Prime Video UK,53200,996
2,UCwQJJVNbn_Tqze89xf83nhw,Entertainment,2007-09-12,http://www.youtube.com/channel/UCwQJJVNbn_Tqze...,The Asylum - Officia...,11200,210
3,UCX8yUJVv_GCpBNw7yp1Y_4Q,Comedy,2006-10-23,http://www.youtube.com/channel/UCX8yUJVv_GCpBN...,Anthony Ingruber,22100,77
4,UC1AJxP0vC_yqPUtr2MQttRQ,People & Blogs,2008-07-27,http://www.youtube.com/channel/UC1AJxP0vC_yqPU...,20thCenturyFoxFilm,44900,477
5,UCi_iW2lqB3ADsJOobe_8yeg,Film and Animation,2011-09-14,http://www.youtube.com/channel/UCi_iW2lqB3ADsJ...,Paramount Pictures U...,468000,976
6,UCFSILgKCKo35QYGz8Kob51g,Film and Animation,2008-01-03,http://www.youtube.com/channel/UCFSILgKCKo35QY...,StudiocanalUK,97200,1844
7,UC_yl-sQID1NrBwAcS1LRHmg,Comedy,2009-11-16,http://www.youtube.com/channel/UC_yl-sQID1NrBw...,Mandatory,39100,1195
8,UCSi5779nMiFTbLm3prHaa1g,Film and Animation,2008-01-15,http://www.youtube.com/channel/UCSi5779nMiFTbL...,Roadshow Films,85400,2138
9,UCNgSkSyLpwJckPsCcpDc2Ow,Entertainment,2012-06-26,http://www.youtube.com/channel/UCNgSkSyLpwJckP...,Adi Shankar's Bootle...,161599,69


In [21]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,http://www.youtube.com/channel/UCISF5OGuAtSLNF...,Magpiepony,736000,304
1,UC94Z4HZJkhPm94YPH1GE3bw,Entertainment,2011-12-13,http://www.youtube.com/channel/UC94Z4HZJkhPm94...,EileMonty,504000,211
2,UCHKDtSVVEc686EW7jRSCpZA,Entertainment,2007-04-01,http://www.youtube.com/channel/UCHKDtSVVEc686E...,Scribbler Production...,327000,1767
3,UC8Wj98MR_oUHBpTjLsE3HuA,Film and Animation,2010-10-10,http://www.youtube.com/channel/UC8Wj98MR_oUHBp...,Pinkie Rose,247000,167
4,UCg3bKR3Z0rCmUe81X06EcaA,Entertainment,2015-11-21,http://www.youtube.com/channel/UCg3bKR3Z0rCmUe...,Blank Slate,121000,408
5,UCPflSzRRyzK2hhbVxNAnrhw,Film and Animation,2012-03-31,http://www.youtube.com/channel/UCPflSzRRyzK2hh...,Pinkie Pie,1229394,451
6,UCcZjYWpM72s-1cinYfeSCag,People & Blogs,2010-08-05,http://www.youtube.com/channel/UCcZjYWpM72s-1c...,DisneyFanatic2364,205000,1026
7,UCcGuE4cmWhB4XLjSKA8eRFA,Entertainment,2014-08-30,http://www.youtube.com/channel/UCcGuE4cmWhB4XL...,WatchPony,117000,341
8,UCaRdKsgbUdMIp-9FwEsaokg,Film and Animation,2014-02-25,http://www.youtube.com/channel/UCaRdKsgbUdMIp-...,Ashley H,77200,500
9,UChU79FLnClqlRSTSZe3EjNA,Film and Animation,2014-03-21,http://www.youtube.com/channel/UChU79FLnClqlRS...,Flutter525,858000,1297


### Channels selected over the comments dataset
We randomly choose 10 000 users over the dataset.
For each user, we then pick two channels at random in the set of channels this user commented in.

In [5]:
'''
# Load the channel tuple sparse matrix
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/jouven/final_sparse_matrix/sparse_matrix_word2vec_users_commented_geq_2_channels.npz')

# Sample NB_SAMPLE from the set of users
NB_SAMPLE = 3000
selected_users = np.random.randint(S.shape[1] - 1, size=NB_SAMPLE)
S = S[:, selected_users]

# Create and store channels tuples
channels_tuple = []
for i in range(S.shape[1]):
    idx = S[:, i].nonzero()
    idx = idx[0]
    
    selected_channels = random.choices(idx, k = 2)
    channels_tuple.append((selected_channels[0], selected_channels[1]))
    
with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_user_walk.pkl",'wb') as f:
     pickle.dump(channels_tuple, f)
f.close()
'''

'\n# Load the channel tuple sparse matrix\nS = scipy.sparse.load_npz(\'/dlabdata1/youtube_large/jouven/final_sparse_matrix/sparse_matrix_word2vec_users_commented_geq_2_channels.npz\')\n\n# Sample NB_SAMPLE from the set of users\nNB_SAMPLE = 3000\nselected_users = np.random.randint(S.shape[1] - 1, size=NB_SAMPLE)\nS = S[:, selected_users]\n\n# Create and store channels tuples\nchannels_tuple = []\nfor i in range(S.shape[1]):\n    idx = S[:, i].nonzero()\n    idx = idx[0]\n    \n    selected_channels = random.choices(idx, k = 2)\n    channels_tuple.append((selected_channels[0], selected_channels[1]))\n    \nwith open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_user_walk.pkl",\'wb\') as f:\n     pickle.dump(channels_tuple, f)\nf.close()\n'

In [10]:
with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_user_walk.pkl",'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

#### Random walk

In [11]:
def get_random_walk(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_random_walk.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

#### Compute metrics: users walk distance and relative nearest neighbor ranking 

In this section we want to measure the euclidian distance of a user walk compared to a random walk.

user walk: Euclidean distance in the embedding space between two randomly channels of a user.
random walk: Euclidean distance in the embedding space between two randomly channels.
position: Position of a channel taken from user u relatively of another channel taken from the same user in terms of its nearest neighbor ranking.


In [12]:
'''
Get the position of ref_channel relative to second_channel in terms of its nearest neighbors ranking.
PARAMETER:
    - ref_channel: The reference channel on which wwe compute it's k nearest neighbor
    - second_channel: The channel where we compute it's ranking relatively to ref_channel
    - dist: Euclidean distance between ref_channel and second_channel
    - index: annoy index
    - df_embedding: DataFrame representing the embedding space

RETURN: The position of second_channel relatively to ref_channel in terms of it's ranking

'''
def get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding):
    
    nearest_neighbors_index = index.get_nns_by_item(ref_channel, len(df_embedding), search_k = 100000000)
    dist_k_th_nearest = distance.euclidean(df_embedding.iloc[ref_channel], 
                                           df_embedding.iloc[nearest_neighbors_index[len(nearest_neighbors_index)-1]])
    for i in range(0, len(nearest_neighbors_index)):
        if nearest_neighbors_index[i] == second_channel:
            return i
    

In [13]:
users_walk_tab = []
ranking_position_tab = []

len_random_set = len(channels_tuple)
len_embedding = len(channels_id)

files = ['channel_embedding_30M_diff_channels_7']
for file in files: 
    print('file ', file)
    df_embedding = get_dataframe_in_embedding_space(file)
    n_comp = df_embedding.shape[1]
    print('n_comp ', n_comp)
    random_walk_distance = get_random_walk(df_embedding)
        
    index = get_annoy_index(df_embedding)

    users_walk = 0
    ranking_position = 0

    for ref_channel, second_channel in channels_tuple:
        dist = distance.euclidean(df_embedding.iloc[ref_channel], df_embedding.iloc[second_channel])
        users_walk += dist
        ranking_position += get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding)
    
    users_walk_tab.append(users_walk/random_walk_distance)
    ranking_position_tab.append(ranking_position/(len_random_set*len_embedding))


file  channel_embedding_30M_diff_channels_7
n_comp  200


In [14]:
users_walk_tab

[2.1226997138720893]

In [15]:
ranking_position_tab

[0.6406605442892896]

In [ ]:
'''
Plot the results obtained when computing the metrics
'''
import plotly.graph_objects as go

components_name = ['Features dim 50', 'Features dim 100', 'Features dim 200', 'Features dim 500']
colors = ['b', 'g,', 'r', 'c']

fig = go.Figure()

for i in range(len(components_name)):
    fig.add_trace(go.Scatter(
        x=ranking_position_tab,
        y=users_walk_tab,
        marker=dict(color=colors[i], size=12),
        mode="markers",
        name=components_name[i],
    ))

fig.update_layout(title="Gender Earnings Disparity",
                  xaxis_title="Ranking position",
                  yaxis_title="Users walk distance")

fig.show()